In [16]:
import csv

def read_csv_file(filename):
    data = {}
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            time = int(row[0])
            value = float(row[1])
            if time not in data:
                data[time] = []
            data[time].append(value)
    return data

def calculate_jackknife_estimations(data):
    jackknife_estimations = {}
    mean_values = {}
    
    for time, values in data.items():
        mean = sum(values) / len(values)
        mean_values[time] = mean
        
        estimations = []
        for idx in range(len(values)):
            excluded_values = values[:idx] + values[idx+1:]
            average = sum(excluded_values) / len(excluded_values)
            estimations.append(average)
        jackknife_estimations[time] = estimations
    
    return mean_values, jackknife_estimations

def write_jackknife_results(filename, jackknife_estimations):
    with open(filename, 'w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Time', 'Jackknife Estimations'])
        for time, estimations in jackknife_estimations.items():
            csv_writer.writerow([time] + estimations)

if __name__ == "__main__":
    input_filename = input("Enter the input CSV filename: ")
    output_filename_jackknife = "jackknife_results.csv"
    output_filename_mean = "mean_results.csv"

    data = read_csv_file(input_filename)
    mean_values, jackknife_estimations = calculate_jackknife_estimations(data)
    write_jackknife_results(output_filename_jackknife, jackknife_estimations)

    with open(output_filename_mean, 'w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Time', 'Mean Value'])
        for time, mean in mean_values.items():
            csv_writer.writerow([time, mean])

    print("Jackknife estimations saved to", output_filename_jackknife)
    print("Mean values saved to", output_filename_mean)


Enter the input CSV filename: 1.csv
Jackknife estimations saved to jackknife_results.csv
Mean values saved to mean_results.csv


In [17]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# Function to get the next available file name based on the current count
def get_next_file_name(base_name, count):
    return f"{base_name}_{count}.csv"

# Prompt for file name
file_name = input("Enter the file name: ")
data = pd.read_csv(file_name)
time = data.iloc[:, 0]
data_values = data.iloc[:, 1]

# Finding  the maximum and minimum values of time
max_time = time.max()
min_time = time.min()
n = max_time
# Calculating the mean value for each unique time
unique_times = np.unique(time)
data_mean = []
for t in unique_times:
    indices = np.where(time == t)[0]
    if len(indices) > 0:
        mean_value = np.mean(data_values.iloc[indices])  # Use .iloc indexing for DataFrame
        data_mean.append(mean_value)

x = unique_times
y = np.array(data_mean)

# Model function
def model_func(x, m, a, b):
    return a * np.exp(-m * x) + b * np.exp(-n* m * x)

# Fit with different windows
num_windows = 50
fit_window_size = 10
start = min_time + 20  

fit_windows = np.linspace(start, len(x) - fit_window_size, num_windows, dtype=int)

# Determining the current count of files with the base name
output_base_name = 'fit_results'
count = 1
while os.path.exists(get_next_file_name(output_base_name, count)):
    count += 1

# Store the results in a list of dictionaries
fit_results = []

for i, window_start in enumerate(fit_windows):
    window_end = window_start + fit_window_size

    # Fit
    params, _ = curve_fit(model_func, x[window_start:window_end], y[window_start:window_end], maxfev=10000)

    m_value, _, _ = params

    # Store the results in a dictionary
    result_dict = {
        'Window Range': f'{window_start}-{window_end}',
        'm': m_value
    }
    fit_results.append(result_dict)

# Saves the results to a CSV file with the current count
output_file_name = get_next_file_name(output_base_name, count)
output_df = pd.DataFrame(fit_results)
output_df.to_csv(output_file_name, index=False)

# Displays the results on the screen
print("Results saved to:", output_file_name)
print("Results:")
print(output_df)


Enter the file name: mean_results.csv
Results saved to: fit_results_25.csv
Results:
   Window Range    m
0         20-30  1.0
1         23-33  1.0
2         26-36  1.0
3         29-39  1.0
4         33-43  1.0
5         36-46  1.0
6         39-49  1.0
7         43-53  1.0
8         46-56  1.0
9         49-59  1.0
10        53-63  1.0
11        56-66  1.0
12        59-69  1.0
13        62-72  1.0
14        66-76  1.0
15        69-79  1.0
16        72-82  1.0
17        76-86  1.0
18        79-89  1.0
19        82-92  1.0
20        86-96  1.0
21        89-99  1.0
22       92-102  1.0
23       96-106  1.0
24       99-109  1.0
25      102-112  1.0
26      105-115  1.0
27      109-119  1.0
28      112-122  1.0
29      115-125  1.0
30      119-129  1.0
31      122-132  1.0
32      125-135  1.0
33      129-139  1.0
34      132-142  1.0
35      135-145  1.0
36      139-149  1.0
37      142-152  1.0
38      145-155  1.0
39      148-158  1.0
40      152-162  1.0
41      155-165  1.0
42      158-1

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
